In [1]:
import requests
import bs4
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import pandas as pd

In [306]:
url_inicial='http://www.revistagastroenterologiamexico.org/es-numeros-anteriores-1995'
url_root= 'http://www.revistagastroenterologiamexico.org/es-numeros-anteriores-1995'
r = requests.get(url_inicial)

In [307]:
soup = BeautifulSoup(r.text, 'html.parser')

In [308]:
box = soup.find('div', class_='contenido')
volumen=box.findAll('div', class_='sumario row')
vol = [x.find('a').get('href')for x in volumen]
vol=[urljoin(url_root,i) for i in vol]
vol

['http://www.revistagastroenterologiamexico.org/es-vol-60-num-4-sumario-X0375090695X67148',
 'http://www.revistagastroenterologiamexico.org/es-vol-60-num-3-sumario-X0375090695X6713X',
 'http://www.revistagastroenterologiamexico.org/es-vol-60-num-2-sumario-X0375090695X67121',
 'http://www.revistagastroenterologiamexico.org/es-vol-60-num-1-sumario-X0375090695X67113']

In [309]:
def get_url_items(sopa, url): 
    box = soup.find('div', class_='contenido')
    volumen=box.findAll('div', class_='sumario row')
    vol = [x.find('a').get('href')for x in volumen]
    vol=[urljoin(url_root,i) for i in vol]
    return vol

In [310]:
links_items=[]
i=0
while i<1:
    i+=1
    print(f'Estoy en la pagina {url_inicial}')
    r_pag=requests.get(url_inicial)
    s_p=BeautifulSoup(r_pag.text,'html.parser')
    links=get_url_items(s_p, url_inicial)
    links_items.append(links) 
    next_a=s_p.select('ul.decada-2022-2020 > a')
    if not next_a or not next_a[0].get('href'):
        break
    url_inicial = urljoin(url_inicial, next_a[0].get('href'))

Estoy en la pagina http://www.revistagastroenterologiamexico.org/es-numeros-anteriores-1995


In [311]:
list_scraper=[]
for i in links_items:
    for j in i:
        list_scraper.append(j)
len(list_scraper)

4

In [312]:
uno=list_scraper[0]
r_item=requests.get(uno)
s_item=BeautifulSoup(r_item.text, 'html.parser')

In [313]:
#funcion para iniciar el escraper de cada libro
def scraper_book(url):
    content_book={}
    r=requests.get(url)
    tire='Revista de Gastroenterología de México'
    a='III. Medicina y Ciencias de la Salud'
    tem='Gastroenterología'
    url='http://www.revistagastroenterologiamexico.org/'
    s_item=BeautifulSoup(r.text,'html.parser')
    #titulo de revista
    try:
        titulo=tire
        content_book['Titulo de revista']=tire
    except AttributeError:
        content_book['Titulo de revista']=None
    #area
    try:
        area=a
        content_book['Área']=a
    except AttributeError:
        content_book['Área']=None
    #tematica
    try:
        tema=tem
        content_book['Temática']=tem
    except AttributeError:
        content_book['Temática']=None
    #titulo libro
    try:
        titu=s_item.find('a', class_='titulo').get_text(strip=True)
        content_book['Titulo del libro']=titu
    except AttributeError:
        content_book['Titulo del libro']=None
    #resumen
    try:
        resu=s_item.find('div', id='articleAbstract').get_text(strip=True)
        content_book['Resumen']=resu
    except AttributeError:
        content_book['Resumen']=None
    #Link incial
    try:
        inicial=url
        content_book['Link inicial']=url
    except AttributeError:
        content_book['Link inicial']=None
    #link articulo
    try:
        link=s_item.find('a', class_='titulo').get('href')
        content_book['Link articulo']=urljoin(url_root, link)
    except AttributeError:
        content_book['Link articulo']=None
    return content_book

In [314]:
list_scraper=list_scraper[0:4]
datos_book=[]
for idx, i in enumerate(list_scraper):
    print(f'estas escrapeando la pag {idx}')
    datos_book.append(scraper_book(i))

estas escrapeando la pag 0
estas escrapeando la pag 1
estas escrapeando la pag 2
estas escrapeando la pag 3


In [315]:
df_catalogo=pd.DataFrame(datos_book)
df_catalogo

,Titulo de revista,Área,Temática,Titulo del libro,Resumen,Link inicial,Link articulo
0,Revista de Gastroenterología de México,III. Medicina y Ciencias de la Salud,Gastroenterología,El cirujano y el tratamiento de los tumores de...,None,http://www.revistagastroenterologiamexico.org/,http://www.revistagastroenterologiamexico.org/...
1,Revista de Gastroenterología de México,III. Medicina y Ciencias de la Salud,Gastroenterología,Cáncer de duodeno,None,http://www.revistagastroenterologiamexico.org/,http://www.revistagastroenterologiamexico.org/...
2,Revista de Gastroenterología de México,III. Medicina y Ciencias de la Salud,Gastroenterología,Helicobacter pylori: úlcera y cáncer gástrico,None,http://www.revistagastroenterologiamexico.org/,http://www.revistagastroenterologiamexico.org/...
3,Revista de Gastroenterología de México,III. Medicina y Ciencias de la Salud,Gastroenterología,Lipodistrofia del mesenterio o paniculitis mes...,None,http://www.revistagastroenterologiamexico.org/,http://www.revistagastroenterologiamexico.org/...


In [316]:
df_catalogo.to_csv('Revista19.28.csv', index=False)